# Python pour le Big Data

Pour ce projet, on se propose d'analyser les relations et causalités de différentes variables tirées de données calculées à partir de données de la blockchain Bitcoin.

In [246]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as plotly_express
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler

## Visualisation des données

### Chargement des données

In [140]:
external_df = pd.read_csv('timeseries/external.csv')
global_df = pd.read_csv('timeseries/global.csv')

# Rename the first column of dates from 'Unnamed: 0' to 'Date'
global_df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
external_df = external_df.drop(columns=['Unnamed: 0']).rename(columns={'date': 'Date'})

# Set the 'Date' column as the index
global_df.set_index('Date', inplace=True)
external_df.set_index('Date', inplace=True)

external_df.head()

,PriceUSD,HashRate
Date,,
2015-01-02,315.942732,333345.017260
2015-01-03,285.647310,327284.198765
2015-01-04,263.334575,347486.927083
2015-01-05,275.003852,315162.561773
2015-01-06,287.549521,319203.107437


En ce qui concerne les données fournies par le fichier `global.csv`, on s'intéressera aux variables suivantes:
* nb_transactions: nombre total de transactions
* nb_payments: nombre total de paiements (un paiement peut être composé de plusieurs transactions)
* nb_mining: Nombre de minages
* newly_created_coins: Nombre de bitcoins créés

Nous allons donc filter les données pour ne garder que ces variables. Il est important de noter que les données représentent des séries temporelles, donc il est nécessaire garder également les données relatives aux dates.

In [141]:
global_df = global_df[['year', 'month', 'day', 'nb_transactions', 'nb_payments', 'nb_mining', 'newly_created_coins']]
global_df.head()

,year,month,day,nb_transactions,nb_payments,nb_mining,newly_created_coins
Date,,,,,,,
2015-01-01,2015,1,1,58882,161194,166,415000000000
2015-01-02,2015,1,2,77439,257633,166,415000000000
2015-01-03,2015,1,3,83644,295618,166,415000000000
2015-01-04,2015,1,4,85893,280288,173,432500000000
2015-01-05,2015,1,5,94862,286552,155,387500000000


Concaténons maintenant les deux dataframes pour créer un 'super' dataframe contenant toutes les données que nous allons utiliser.

In [166]:
# Concatenate the two dataframes
df = pd.concat([global_df, external_df], axis=1)
df.head()

,year,month,day,nb_transactions,nb_payments,nb_mining,newly_created_coins,PriceUSD,HashRate
Date,,,,,,,,,
2015-01-01,2015,1,1,58882,161194,166,415000000000,NaN,NaN
2015-01-02,2015,1,2,77439,257633,166,415000000000,315.942732,333345.017260
2015-01-03,2015,1,3,83644,295618,166,415000000000,285.647310,327284.198765
2015-01-04,2015,1,4,85893,280288,173,432500000000,263.334575,347486.927083
2015-01-05,2015,1,5,94862,286552,155,387500000000,275.003852,315162.561773


### Prétraitement des données (normalisation, lissage, etc.)

In [170]:
# Save the dataframe indices and columns
columns = df.columns
indices = df.index

# Normalize the data
normalized_df = StandardScaler().fit_transform(df)
normalized_df = pd.DataFrame(np.array(normalized_df, dtype=np.float64), columns=columns, index=indices)

# Data smoothing
normalized_df = pd.DataFrame(normalized_df).rolling(7).mean()

### Affichage des données

Affichons les données par paires pour gagner en compréhension.

In [275]:
fig = plotly_express.line(normalized_df, x=normalized_df.index, y=['PriceUSD', 'HashRate'], title='Bitcoin price and hashrate and blockchain activity')
fig.show()

fig = plotly_express.line(normalized_df, x=normalized_df.index, y=['nb_transactions', 'nb_payments'], title='Bitcoin transactions and payments')
fig.show()

fig = plotly_express.line(normalized_df, x=normalized_df.index, y=['nb_mining', 'newly_created_coins'], title='Bitcoin mining and newly created coins')
fig.show()